create_smartcart_db.ipynb

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import pandas as pd
import ast
import sqlite3
from pprint import pprint


In [5]:
from IPython.display import clear_output
!pip install vit_keras
clear_output()

In [6]:
import os

folder_path = "/content/drive/MyDrive/dataset11"
folders = os.listdir(folder_path)

# This shows hidden characters like \u200b
for f in folders:
    print(repr(f))


'[use case 1] Inventory Optimization for Retail\u200b\u200b'
'[Usecase 5] AI-Powered Job Application Screening System\u200b'
'[Usecase 4] AI for Elderly Care and Support\u200b'
'[Usecase 7] AI-Driven Customer Support Enhancing Efficiency Through Multiagents\u200b'
'[Usecase 2] Personalized Recommendations for E-Commerce\u200b\u200b'
'[Usecase 3] AI for Sustainable Agriculture\u200b'


In [7]:
usecase2_folder = "[Usecase 2] Personalized Recommendations for E-Commerce\u200b\u200b"
data_path = f"/content/drive/MyDrive/dataset11/{usecase2_folder}"


In [8]:
import pandas as pd

df_customer = pd.read_csv(f"{data_path}/customer_data_collection.csv")
df_product = pd.read_csv(f"{data_path}/product_recommendation_data.csv")


In [9]:
df_customer['Browsing_History'] = df_customer['Browsing_History'].apply(ast.literal_eval)
df_customer['Purchase_History'] = df_customer['Purchase_History'].apply(ast.literal_eval)

customer_profiles = {}
for _, row in df_customer.iterrows():
    customer_id = row['Customer_ID']
    browsing = row['Browsing_History']
    purchases = row['Purchase_History']

    interest_keywords = browsing + purchases
    profile = {
        "Location": row['Location'],
        "Segment": row['Customer_Segment'],
        "Top_Interests": list(set(interest_keywords)),
        "Avg_Order_Value": row['Avg_Order_Value'],
        "Season": row['Season'],
        "Holiday_Shopper": row['Holiday']
    }
    customer_profiles[customer_id] = profile


In [10]:
df_product['Similar_Product_List'] = df_product['Similar_Product_List'].apply(ast.literal_eval)

product_profiles = {}
for _, row in df_product.iterrows():
    product_id = row['Product_ID']
    profile = {
        "Category": row['Category'],
        "Subcategory": row['Subcategory'],
        "Brand": row['Brand'],
        "Price": row['Price'],
        "Product_Rating": row['Product_Rating'],
        "Sentiment_Score": row['Customer_Review_Sentiment_Score'],
        "Season": row['Season'],
        "Holiday": row['Holiday'],
        "Geo_Location": row['Geographical_Location'],
        "Similar_Products": row['Similar_Product_List'],
        "Recommendation_Probability": row['Probability_of_Recommendation']
    }
    product_profiles[product_id] = profile


In [11]:
def recommend_products_for_customer(customer_id, top_n=3):
    customer = customer_profiles[customer_id]
    recommendations = []

    for pid, product in product_profiles.items():
        score = 0
        if product['Subcategory'] in customer['Top_Interests'] or product['Category'] in customer['Top_Interests']:
            score += 2
        if product['Season'] == customer['Season']:
            score += 1
        if product['Recommendation_Probability'] > 0.5:
            score += 1
        if product['Sentiment_Score'] > 0.5:
            score += 1
        recommendations.append((pid, score))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]


In [12]:
conn = sqlite3.connect('smartcart.db')
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS customer_profiles")
cursor.execute("DROP TABLE IF EXISTS product_profiles")
cursor.execute("DROP TABLE IF EXISTS recommendations")

cursor.execute('''
CREATE TABLE customer_profiles (
    Customer_ID TEXT PRIMARY KEY,
    Location TEXT,
    Segment TEXT,
    Top_Interests TEXT,
    Avg_Order_Value REAL,
    Season TEXT,
    Holiday_Shopper TEXT
)
''')

cursor.execute('''
CREATE TABLE product_profiles (
    Product_ID TEXT PRIMARY KEY,
    Category TEXT,
    Subcategory TEXT,
    Brand TEXT,
    Price REAL,
    Product_Rating REAL,
    Sentiment_Score REAL,
    Season TEXT,
    Holiday TEXT,
    Geo_Location TEXT,
    Similar_Products TEXT,
    Recommendation_Probability REAL
)
''')

cursor.execute('''
CREATE TABLE recommendations (
    Customer_ID TEXT,
    Product_ID TEXT,
    Score INTEGER
)
''')
conn.commit()


In [13]:
# Insert customers
for cid, profile in customer_profiles.items():
    cursor.execute('''
        INSERT INTO customer_profiles
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        cid,
        profile['Location'],
        profile['Segment'],
        ', '.join(profile['Top_Interests']),
        profile['Avg_Order_Value'],
        profile['Season'],
        profile['Holiday_Shopper']
    ))

# Insert products
for pid, profile in product_profiles.items():
    cursor.execute('''
        INSERT INTO product_profiles
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        pid,
        profile['Category'],
        profile['Subcategory'],
        profile['Brand'],
        profile['Price'],
        profile['Product_Rating'],
        profile['Sentiment_Score'],
        profile['Season'],
        profile['Holiday'],
        profile['Geo_Location'],
        ', '.join(profile['Similar_Products']),
        profile['Recommendation_Probability']
    ))

# Insert recommendations
for cid in customer_profiles:
    top_recs = recommend_products_for_customer(cid, top_n=3)
    for pid, score in top_recs:
        cursor.execute('''
            INSERT INTO recommendations
            VALUES (?, ?, ?)
        ''', (cid, pid, score))

conn.commit()


In [15]:
cursor.execute("SELECT * FROM recommendations LIMIT 5")
print(cursor.fetchall())


[('C1000', 'P2005', 5), ('C1000', 'P2015', 5), ('C1000', 'P2059', 5), ('C1001', 'P2086', 5), ('C1001', 'P2111', 5)]


In [14]:
from google.colab import files
files.download('smartcart.db')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>